In [2]:
%load_ext autoreload
%autoreload 2

import os
import sys
module_path = os.path.abspath('RS/thinker/thinker')
if module_path not in sys.path:
    sys.path.append(module_path)

from collections import namedtuple
from matplotlib import pyplot as plt
import matplotlib.ticker as mticker
from collections import deque
import time
import numpy as np
import argparse
import torch
import torch.nn.functional as F
from thinker.env import Environment
from thinker.net import ActorNet, ModelNet
from thinker.buffer import ModelBuffer
import thinker.util as util

flags = util.parse([])
flags.model_batch_size = 2
flags.model_buffer_n = 1000
flags.model_warm_up_n = 500
flags.model_batch_mode = False
flags.model_rnn = True
flags.model_unroll_length = 8
t = flags.model_unroll_length   
flags.model_k_step_return = 5
k = flags.model_k_step_return
flags.actor_parallel_n = 1
n = flags.actor_parallel_n  

flags.model_batch_mode = True

P = namedtuple("P", ["x","y"])
model_buffer = ModelBuffer(flags)

c = 0
for c in range(100):
    data = P(torch.full((t+k, n, 1),c), torch.full((t+k, n, 1),c+0.1))    
    state = (torch.full((n, 3),c+0.2), torch.full((n, 3),c+0.3))
    model_buffer.write(data, state, np.random.randint(10))

print(model_buffer.next_inds)
data, data_state, weights, abs_flat_inds, inds = model_buffer.read(1)
print(data.x[:, :, -1], data_state)

state = tuple(torch.zeros_like(x) for x in data_state)
priorities = torch.zeros(len(abs_flat_inds))

model_buffer.update_priority(abs_flat_inds, priorities, state)


TypeError: Actors cannot be instantiated directly. Instead of 'ModelBuffer()', use 'ModelBuffer.remote()'.

In [82]:
model_buffer.state[int(62-model_buffer.base_ind/n)]

(tensor([[62.2000, 62.2000, 62.2000]]), tensor([[62.3000, 62.3000, 62.3000]]))

In [7]:
from thinker.env import Environment
from thinker.net import ModelNet

flags = util.parse([])
flags.model_rnn = False

model_net = ModelNet((3,80,80), 5, flags)
print(model_net.rnn)

env = Environment(flags)
env_out, model_state = env.initial(model_net)

a = torch.tensor([1,1,1]).long().unsqueeze(0).unsqueeze(0)
for n in range(1, 40 * 150):
    env_out, model_state = env.step(a, model_net, model_state)
    print(env_out.episode_step, model_state)

False
tensor([[1]], dtype=torch.int32) None
tensor([[2]], dtype=torch.int32) None
tensor([[3]], dtype=torch.int32) None
tensor([[4]], dtype=torch.int32) None
tensor([[5]], dtype=torch.int32) None
tensor([[6]], dtype=torch.int32) None
tensor([[7]], dtype=torch.int32) None
tensor([[8]], dtype=torch.int32) None
tensor([[9]], dtype=torch.int32) None
tensor([[10]], dtype=torch.int32) None
tensor([[11]], dtype=torch.int32) None
tensor([[12]], dtype=torch.int32) None
tensor([[13]], dtype=torch.int32) None
tensor([[14]], dtype=torch.int32) None
tensor([[15]], dtype=torch.int32) None
tensor([[16]], dtype=torch.int32) None
tensor([[17]], dtype=torch.int32) None
tensor([[18]], dtype=torch.int32) None
tensor([[19]], dtype=torch.int32) None
tensor([[20]], dtype=torch.int32) None
tensor([[21]], dtype=torch.int32) None
tensor([[22]], dtype=torch.int32) None
tensor([[23]], dtype=torch.int32) None
tensor([[24]], dtype=torch.int32) None
tensor([[25]], dtype=torch.int32) None
tensor([[26]], dtype=torch.i